# Modélisation et évaluation des modèles

**Auteur :** Mehdi MUNIM

**Date :** 2023-11-26

**Description :**

Ce notebook entraîne et évalue différents modèles de Machine Learning pour la classification de sentiment sur les données de tweets prétraitées.

In [ ]:
#@title 1. Importation des librairies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import sys

# Ajouter le chemin du dossier src pour importer les modules
sys.path.append('../src')
import model
import evaluation

## 2. Chargement des données

In [ ]:
#@title 2.1 Chargement des données

# Chemin d'accès au dossier data
data_path = '../data/processed/'

# Charger les données prétraitées
X_train_tfidf = pickle.load(open(data_path + 'X_train_tfidf.pickle', 'rb'))
X_test_tfidf = pickle.load(open(data_path + 'X_test_tfidf.pickle', 'rb'))
X_train_w2v = pickle.load(open(data_path + 'X_train_word2vec.pickle', 'rb'))
X_test_w2v = pickle.load(open(data_path + 'X_test_word2vec.pickle', 'rb'))
X_train_d2v = pickle.load(open(data_path + 'X_train_doc2vec.pickle', 'rb'))
X_test_d2v = pickle.load(open(data_path + 'X_test_doc2vec.pickle', 'rb'))
y_train = pickle.load(open(data_path + 'y_train.pickle', 'rb'))
y_test = pickle.load(open(data_path + 'y_test.pickle', 'rb'))

## 3. Entraînement et évaluation des modèles

In [ ]:
#@title 3.1 Entraînement et évaluation du modèle de régression logistique

from sklearn.linear_model import LogisticRegression

# Créer et entraîner le modèle de régression logistique
model_lr = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
model_lr.fit(X_train_tfidf, y_train)

# Évaluer le modèle
evaluation.evaluer_modele(model_lr, X_test_tfidf, y_test)

In [ ]:
#@title 3.2 Entraînement et évaluation du modèle MLP

# Créer et entraîner le modèle MLP
model_mlp = model.create_mlp_model(input_shape=(X_train_tfidf.shape[1],))
model_mlp.fit(X_train_tfidf, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Évaluer le modèle
evaluation.evaluer_modele(model_mlp, X_test_tfidf, y_test)

In [ ]:
#@title 3.3 Entraînement et évaluation du modèle CNN

# Reshape les données pour le modèle CNN
X_train_cnn = np.reshape(X_train_w2v, (X_train_w2v.shape[0], X_train_w2v.shape[1], 1))
X_test_cnn = np.reshape(X_test_w2v, (X_test_w2v.shape[0], X_test_w2v.shape[1], 1))

# Créer et entraîner le modèle CNN
model_cnn = model.create_cnn_model(input_shape=(X_train_cnn.shape[1], 1))
model_cnn.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Évaluer le modèle
evaluation.evaluer_modele(model_cnn, X_test_cnn, y_test)

In [ ]:
#@title 3.4 Entraînement et évaluation du modèle LSTM

# Créer et entraîner le modèle LSTM
model_lstm = model.create_lstm_model(input_shape=(X_train_w2v.shape[1],))
model_lstm.fit(X_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Évaluer le modèle
evaluation.evaluer_modele(model_lstm, X_test_w2v, y_test)

In [ ]:
#@title 3.5 Entraînement et évaluation du modèle BERT

# Créer et entraîner le modèle BERT
model_bert, tokenizer = model.create_bert_model()

# Tokenizer les tweets d'entraînement et de test
X_train_bert = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors="tf")
X_test_bert = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors="tf") 1 

# Convertir les labels en tenseurs
y_train_bert = tf.convert_to_tensor(y_train)
y_test_bert = tf.convert_to_tensor(y_test)

# Entraîner le modèle
history = model_bert.fit(
    x={"input_ids": X_train_bert["input_ids"], "attention_mask": X_train_bert["attention_mask"]},
    y=y_train_bert,
    validation_split=0.2,
    epochs=3,
    batch_size=8,
)

# Évaluer le modèle
y_pred_bert = model_bert.predict(X_test_bert).logits
y_pred_bert = np.argmax(y_pred_bert, axis=1)
evaluation.evaluer_modele(model_bert,
                          {"input_ids": X_test_bert["input_ids"], "attention_mask": X_test_bert["attention_mask"]},
                          y_test_bert)

In [ ]:
#@title 3.6 Entraînement et évaluation du modèle USE

import tensorflow_hub as hub

# Charger le modèle USE
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Encoder les tweets
X_train_use = embed(X_train)
X_test_use = embed(X_test)

# Créer et entraîner le modèle USE
model_use = model.create_use_model(input_shape=(X_train_use.shape[1],))
model_use.fit(X_train_use, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Évaluer le modèle
evaluation.evaluer_modele(model_use, X_test_use, y_test)

## 4. Comparaison des modèles

In [ ]:
#@title Comparaison des modèles
# Fonction pour comparer les modèles en utilisant les métriques d'évaluation
def comparer_modeles(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return accuracy, precision, recall, f1

# Comparaison des modèles
modeles = [model_baseline, model_mlp, model_cnn, model_lstm]
noms_modeles = ['Régression Logistique', 'MLP', 'CNN', 'LSTM']
y_preds = [model.predict(X_test_kbest) for model in modeles]

# Calculer les métriques pour chaque modèle
resultats = []
for y_pred in y_preds:
    resultats.append(comparer_modeles(y_test, y_pred))

# Créer un DataFrame pour afficher les résultats
comparaison = pd.DataFrame(resultats, columns=['Accuracy', 'Precision', 'Recall', 'F1-score'], index=noms_modeles)

# Afficher les résultats
print(comparaison)